# Homework Assignment 5
## by Kavisha Reddy

## Links to repositories and webpage:
1. Homework repo: https://github.com/Kavisha-29-R/homework-5-Kavisha-29-R.git
2. Webpage repo: https://github.com/Kavisha-29-R/hw5-Sankey-Plot.git 
## Live Sankey Diagram
You can view the interactive Sankey diagram for Subject 127 [Sankey-Diagram](https://kavisha-29-r.github.io/hw5-Sankey-Plot/sankey.html)


### Instructions:

Submit a PDF or HTML export of your hw5.ipnyb file. Before exporting, select "Restart & Run All" to make sure all code runs cleanly and outputs are displayed. Also submit both .html plots.

1. Create a python file that webscrapes GDP by country [Wiki - GDP](https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)). and plots a stacked interactive bar plot using plotly. 
    - Stack countries within regions using the IMF numbers. 
    - Please include this in your ipython notebook and output your plot to an html file containing the plot.
2. Look at the chapter on interactive graphics [CH - interactive graphic desgin](https://smart-stats.github.io/ds4bio_book/book/_build/html/interactive.html). and, specifically, the code to display a subject's MRICloud data as a sunburst plot. 
Do the following: 
    - Display this subject's data as a Sankey diagram [Sankey diagram](https://plotly.com/python/sankey-diagram/). 
    - Display as many levels as you can (at least 3) for Type = 1, starting from the intracranial volume.
3. Create a simple webpage containing the Sankey graphic and host it on github pages. 
    - Do not - host this off of your assignment repo from github classroom, since this is not public. Instead, you'll have to create a new public repo from your regular github account and add this file. 
    - Put the link to your live web page in a markdown cell of your hw5.ipynb file as a text block.

Your homework should include:
* An file called hw5.ipynb that has your code for parts 1 and 2.
* Two html files, one called sankey.html and one called stacked_bar.html that contain the two plots as html files.
* Your hw5.ipynb file should have a text block that contains a link to the live and publiclyl hosted sankey diagram.

Remember that you should have two repositories for this assignment. 
- First, you need the HW repository that you create when you accept the assignment. This is where you do the work for the three files (hw5.ipynb, sankey.html, stacked_bar.html). 
- Secondly, you will need to create your own repository containing a live link to your sankey html file. So, when I click on that link it should show a page containing your plot.

Note plotly objects contain a method called to_html() which is useful for creating an html file.



### 1. Web-scrape GDP by country (IMF numbers) and stacked interactive bar plot

In [1]:
# Standard imports used in both parts
import os
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import plotly.express as px
import plotly.colors as pc
import plotly.graph_objects as go

# Optional helper: country_converter (nice for mapping country -> continent/UN region)
try:
    import country_converter as coco   
except Exception:
    coco = None
    print("country_converter not available: region mapping will fall back on a small manual map (see notes).")


In [2]:
# Download the Wikipedia page with a browser-like User-Agent header
wiki_url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0.0.0 Safari/537.36"
}

resp = requests.get(wiki_url, headers=headers)
resp.raise_for_status()  # stop if page couldn't be fetched

# BeautifulSoup parse
soup = BeautifulSoup(resp.text, "html.parser")

# Extract all wikitables
tables = soup.find_all("table", {"class": "wikitable"})
len(tables)
# Use pandas.read_html directly with the headers (need to pass resp.text)
#tables = pd.read_html(resp.text)
print(f"Found {len(tables)} tables")


Found 1 tables


In [3]:
# Convert the first big table to pandas DataFrame
imf_df = pd.read_html(str(tables[0]))[0]

# Clean column names (remove footnotes)
imf_df.columns = [c.split("[")[0].strip() for c in imf_df.columns]

# Rename the country column for clarity
if "Country/Territory" in imf_df.columns:
    imf_df.rename(columns={"Country/Territory": "Country"}, inplace=True)

# Show first few rows
imf_df.head()


C:\Users\Kavisha\AppData\Local\Temp\ipykernel_31048\964544547.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  imf_df = pd.read_html(str(tables[0]))[0]


,Country,IMF (2025),World Bank (2022–24),United Nations (2023)
0,World,113795678,111326370,100834796
1,United States,30507217,29184890,27720700
2,China[n 1],19231705,18743803,17794782
3,Germany,4744804,4659929,4525704
4,India,4187017,3912686,3575778


In [4]:
# Pick only Country + IMF column
# (the IMF column name might be like 'IMF (2025)')
imf_col = [c for c in imf_df.columns if "IMF" in c][0]

gdp_df = imf_df[["Country", imf_col]].copy()

# Filter out 'World' or aggregates
exclude = [
    "World", "World total", "European Union", "Eurozone",
    "Asia", "Africa", "Europe", "Americas", "Oceania"
]
gdp_df = gdp_df[~gdp_df["Country"].isin(exclude)]

# Clean GDP values (remove commas etc.)
gdp_df[imf_col] = (
    gdp_df[imf_col]
    .astype(str)
    .str.replace(",", "", regex=False)
    .str.replace(r"[^0-9.]", "", regex=True)
    .replace("", None)
    .astype(float)
)

# Drop rows with no IMF data
gdp_df = gdp_df.dropna(subset=[imf_col]).reset_index(drop=True)

gdp_df.head(10)


,Country,IMF (2025)
0,United States,30507217.0
1,China[n 1],19231705.0
2,Germany,4744804.0
3,India,4187017.0
4,Japan,4186431.0
5,United Kingdom,3839180.0
6,France,3211292.0
7,Italy,2422855.0
8,Canada,2225341.0
9,Brazil,2125958.0


In [5]:
# normalize column names and extract GDP value column
df = gdp_df.copy()
# lower-case column names for easier matching
df.columns = [str(c).strip() for c in df.columns]

# Try to find a numeric GDP column (some variants: 'GDP (US$million)', 'US$million', 'GDP (millions of US$)')
gdp_col = None
for c in df.columns:
    lc = str(c).lower()
    if "gdp" in lc and ("$" in lc or "us" in lc or "million" in lc or "nominal" in lc):
        gdp_col = c
        break
# If still not found, fall back to any numeric-like column
if gdp_col is None:
    # try to find first numeric-like column (contains digits or commas)
    for c in df.columns:
        sample = df[c].astype(str).head(10).str.replace(",", "").str.replace(".", "", 1)
        if sample.str.isnumeric().any():
            gdp_col = c
            break

if gdp_col is None:
    raise RuntimeError("Could not find a GDP column automatically. Inspect df.columns to pick the correct column.")

print("Detected GDP column:", gdp_col)

# Standardize GDP values to numeric (in US$ millions) by removing commas/footnotes and coercing
def to_numeric_money(series):
    # Remove bracket footnotes like [1], commas, parentheses, and non-numeric characters
    cleaned = series.astype(str).str.replace(r"\[.*?\]", "", regex=True)
    cleaned = cleaned.str.replace(",", "", regex=False)
    cleaned = cleaned.str.replace(r"[^\d.\-]", "", regex=True)
    return pd.to_numeric(cleaned, errors="coerce")

df["country"] = df.iloc[:,0].astype(str).str.replace(r"\[.*\]", "", regex=True).str.strip()
df["gdp_million_usd"] = to_numeric_money(df[gdp_col])

# drop rows without GDP numeric values
df = df.dropna(subset=["gdp_million_usd"]).reset_index(drop=True)
df = df[["country", "gdp_million_usd"]].copy()
df.head(10)


Detected GDP column: IMF (2025)


,country,gdp_million_usd
0,United States,30507217.0
1,China,19231705.0
2,Germany,4744804.0
3,India,4187017.0
4,Japan,4186431.0
5,United Kingdom,3839180.0
6,France,3211292.0
7,Italy,2422855.0
8,Canada,2225341.0
9,Brazil,2125958.0


In [6]:
# Map country -> region (IMF regions or at least continent). We try to use country_converter if available,
# otherwise we use a small manual mapping / fallbacks.

def map_country_to_region(country_name):
    # First try country_converter if available
    if coco is not None:
        try:
            # convert to continent name (e.g., 'Africa', 'Europe')
            cont = coco.convert(names=country_name, to='continent')
            if cont is not None and cont != "not found":
                return cont
        except Exception:
            pass
    # Manual quick-fallbacks (extend if you see missing countries)
    fallback_map = {
        "United States": "North America",
        "United States of America": "North America",
        "Russia": "Europe/Asia",
        "Bolivia": "South America",
        "Congo": "Africa",
        "DR Congo": "Africa",
        "Côte d'Ivoire": "Africa",
        "Ivory Coast": "Africa",
        "South Korea": "Asia",
        "North Korea": "Asia",
        "Laos": "Asia",
        "Vietnam": "Asia",
        "Syria": "Asia",
        "Palestine": "Asia",
        "Czechia": "Europe",
        "Czech Republic": "Europe",
    }
    # try exact
    if country_name in fallback_map:
        return fallback_map[country_name]
    # try partial matching (simple)
    lname = country_name.lower()
    if "united states" in lname or lname == "usa":
        return "North America"
    if "korea" in lname:
        return "Asia"
    if "china" in lname or "india" in lname or "japan" in lname:
        return "Asia"
    if "island" in lname or "seychelles" in lname or "mauritius" in lname:
        return "Africa"
    # last resort: Unknown
    return "Other"

# apply mapping
df['region'] = df['country'].apply(map_country_to_region)

# Show sample of countries flagged "Other" so you can refine mapping if needed
others = df[df['region']=="Other"]
print("Countries mapped to 'Other' (you may wish to add to fallback_map):")
print(others['country'].head(20).to_list())


Countries mapped to 'Other' (you may wish to add to fallback_map):
[]


In [7]:
# For readability: pick the top N countries per region to include in the stacked bar.
# If you want *all* countries you can skip the top-N filter, but stacks become unreadable.
TOP_N = 10  # top 10 countries per region

top_countries = (
    df
    .sort_values(["region","gdp_million_usd"], ascending=[True, False])
    .groupby("region")
    .head(TOP_N)
    .reset_index(drop=True)
)

# We also compute a small 'Other' bucket per region (sum of remaining countries)
others_summary = (
    df
    .merge(top_countries[['country']], on='country', how='left', indicator=True)
    .query("_merge == 'left_only'")
    .groupby('region', as_index=False)
    .agg(country=('country', lambda s: 'Other (rest)'),
         gdp_million_usd=('gdp_million_usd','sum'))
)

# combine top countries + other buckets
plot_df = pd.concat([top_countries[['region','country','gdp_million_usd']], others_summary[['region','country','gdp_million_usd']]], ignore_index=True)
plot_df = plot_df.sort_values(['region','gdp_million_usd'], ascending=[True, False]).reset_index(drop=True)

plot_df.head(30)


,region,country,gdp_million_usd
0,Africa,Other (rest),921038.0
1,Africa,South Africa,410338.0
2,Africa,Egypt,347342.0
3,Africa,Algeria,268885.0
4,Africa,Nigeria,188271.0
5,Africa,Morocco,165835.0
6,Africa,Kenya,131673.0
7,Africa,Ethiopia,117457.0
8,Africa,Angola,113343.0
9,Africa,Ivory Coast,94483.0


In [8]:
# create a stacked bar where each region is on the x-axis and stacks are countries' GDP contributions.
# We pivot the data so that each country is a color segment.
pivot = plot_df.pivot_table(
    index='region',
    columns='country',
    values='gdp_million_usd',
    aggfunc='sum',
    fill_value=0
)

# reshape pivot into long format for Plotly
long_df = pivot.reset_index().melt(id_vars='region', var_name='country', value_name='gdp_million_usd')

# make a large gradient palette: sample Viridis at N points
n_countries = long_df['country'].nunique()
color_scale = pc.sample_colorscale("Rainbow", [i/(n_countries-1) for i in range(n_countries)])

# build stacked bar
fig = px.bar(
    long_df,
    x='region',
    y='gdp_million_usd',
    color='country',
    title="Regional GDP by country (IMF numbers; each bar = region, stacks = countries)",
    labels={"gdp_million_usd": "GDP (million USD)", "region": "Region"},
    barmode='relative',
    color_discrete_sequence=color_scale
)

# update layout for readability
fig.update_layout(
    xaxis_title="Region",
    yaxis_title="GDP (million USD)",
    legend_title="Country",
    template="plotly_white",
    height=600,
)

# save interactive HTML file
stacked_bar_filename = "stacked_bar.html"
fig.write_html(stacked_bar_filename, full_html=True, include_plotlyjs='cdn')
print(f"Saved stacked bar to {stacked_bar_filename}")

# if running in a notebook, show the figure inline as well
fig.show()


Saved stacked bar to stacked_bar.html


### 2. MRICloud-style hierarchy → Sankey diagram (Type == 1, at least 3 levels starting from Intracranial Volume)

#### From book:

In [9]:
## load in the hierarchy information
url = "https://raw.githubusercontent.com/bcaffo/MRIcloudT1volumetrics/master/inst/extdata/multilevel_lookup_table.txt"
multilevel_lookup = pd.read_csv(url, sep = "\t").drop(['Level5'], axis = 1)
multilevel_lookup = multilevel_lookup.rename(columns = {
    "modify"   : "roi",
    "modify.1" : "level4",
    "modify.2" : "level3",
    "modify.3" : "level2",
    "modify.4" : "level1"})
multilevel_lookup = multilevel_lookup[['roi', 'level4', 'level3', 'level2', 'level1']]
multilevel_lookup.head()

,roi,level4,level3,level2,level1
0,SFG_L,SFG_L,Frontal_L,CerebralCortex_L,Telencephalon_L
1,SFG_R,SFG_R,Frontal_R,CerebralCortex_R,Telencephalon_R
2,SFG_PFC_L,SFG_L,Frontal_L,CerebralCortex_L,Telencephalon_L
3,SFG_PFC_R,SFG_R,Frontal_R,CerebralCortex_R,Telencephalon_R
4,SFG_pole_L,SFG_L,Frontal_L,CerebralCortex_L,Telencephalon_L


In [10]:
## Now load in the subject data
id = 127
subjectData = pd.read_csv("https://raw.githubusercontent.com/smart-stats/ds4bio_book/main/book/assetts/kirby21AllLevels.csv")
subjectData = subjectData.loc[(subjectData.type == 1) & (subjectData.level == 5) & (subjectData.id == id)]
subjectData = subjectData[['roi', 'volume']]
## Merge the subject data with the multilevel data
subjectData = pd.merge(subjectData, multilevel_lookup, on = "roi")
subjectData = subjectData.assign(icv = "ICV")
subjectData = subjectData.assign(comp = subjectData.volume / np.sum(subjectData.volume))
subjectData.head()

,roi,volume,level4,level3,level2,level1,icv,comp
0,SFG_L,12926,SFG_L,Frontal_L,CerebralCortex_L,Telencephalon_L,ICV,0.009350
1,SFG_R,10050,SFG_R,Frontal_R,CerebralCortex_R,Telencephalon_R,ICV,0.007270
2,SFG_PFC_L,12783,SFG_L,Frontal_L,CerebralCortex_L,Telencephalon_L,ICV,0.009247
3,SFG_PFC_R,11507,SFG_R,Frontal_R,CerebralCortex_R,Telencephalon_R,ICV,0.008324
4,SFG_pole_L,3078,SFG_L,Frontal_L,CerebralCortex_L,Telencephalon_L,ICV,0.002227


In [11]:
fig = px.sunburst(subjectData, path=['icv', 'level1', 'level2', 'level3', 'level4', 'roi'],
                  values='comp', width=800, height=800)
fig.show()

In [12]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# --- Load subject data and multilevel lookup ---
id = 127
subjectData = pd.read_csv("https://raw.githubusercontent.com/smart-stats/ds4bio_book/main/book/assetts/kirby21AllLevels.csv")

# Filter for Type=1, Level=5, specific subject
subjectData = subjectData.loc[(subjectData.type == 1) & (subjectData.level == 5) & (subjectData.id == id)]
subjectData = subjectData[['roi', 'volume']]

# Columns: roi, level1, level2, level3
subjectData = pd.merge(subjectData, multilevel_lookup, on="roi")

# Add ICV root
subjectData['icv'] = 'ICV'
subjectData['comp'] = subjectData['volume'] / np.sum(subjectData['volume'])

# --- Prepare Sankey labels ---
hierarchy_cols = ['icv', 'level1', 'level2', 'level3']
labels = pd.unique(subjectData[hierarchy_cols].values.ravel())
label_indices = {label: i for i, label in enumerate(labels)}

sources = []
targets = []
values = []
link_colors = []

# Define colors per level
level_colors = {
    'icv': 'pink',
    'level1': 'purple',
    'level2': 'green',
    'level3': 'blue'
}

# Build links and assign color based on source node
for i in range(len(hierarchy_cols) - 1):
    src_col = hierarchy_cols[i]
    tgt_col = hierarchy_cols[i + 1]
    src = subjectData[src_col].map(label_indices)
    tgt = subjectData[tgt_col].map(label_indices)
    val = subjectData['comp']
    sources.append(src)
    targets.append(tgt)
    values.append(val)
    
    # Assign link color same as source node
    colors = []
    for s in src:
        node_label = labels[s]
        if node_label in subjectData['icv'].values:
            colors.append(level_colors['icv'])
        elif node_label in subjectData['level1'].values:
            colors.append(level_colors['level1'])
        elif node_label in subjectData['level2'].values:
            colors.append(level_colors['level2'])
        else:
            colors.append(level_colors['level3'])
    link_colors.append(pd.Series(colors))

# Concatenate all levels
sources_all = pd.concat(sources)
targets_all = pd.concat(targets)
values_all = pd.concat(values)
link_colors_all = pd.concat(link_colors)

# --- Plot Sankey diagram ---
fig = go.Figure(go.Sankey(
    valueformat=".2%",
    valuesuffix="",
    node=dict(
        pad=20,
        thickness=25,
        line=dict(color="black", width=0.5),
        label=labels,
        color=[level_colors['icv'] if l=='ICV' else 
               level_colors['level1'] if l in subjectData['level1'].values else
               level_colors['level2'] if l in subjectData['level2'].values else
               level_colors['level3'] for l in labels]
    ),
    link=dict(
        source=sources_all,
        target=targets_all,
        value=values_all,
        color=link_colors_all
    )
))

fig.update_layout(
    hovermode='x',
    title=dict(text=f"Sankey Diagram: Subject {id} Type=1", font=dict(size=10, color='black')),
    font=dict(size=12, color='black'),
    plot_bgcolor='white',
    paper_bgcolor='white',
    width=1000,
    height=800
)

fig.show()


In [13]:
# Save the Sankey plot as an HTML file
fig.write_html("sankey.html")